# Ejercicio 1 - Punto de alquiler de vehículos

## Enunciado

> Un punto de alquiler de vehículos opera continuamente (24/7) y recibe solicitudes de alquiler de acuerdo con un **proceso de Poisson** con tasa $\lambda$ por hora. El punto cuenta con **50 vehículos** para alquilar y se estima que un **alquiler** toma un **tiempo exponencial** con **media $m$ días**. Los tiempos de alquiler son independientes entre sí y también son independientes de los tiempos entre solicitudes de alquiler. Usted se encuentra analizando el desempeño de este punto de alquiler, para lo cual debe resolver las siguientes preguntas:
>
> **a.** Provea una expresión que permita calcular la **probabilidad** de que en **un día** de operación **lleguen menos de 5** solicitudes de alquiler.
> **b.** Provea una expresión que permita calcular la **probabilidad** de que el **tiempo entre la primera y la segunda** solicitud de alquiler sea **menor a una hora**.
> **c.** Modele este sistema empleando una **cadena de Markov**. Exprese claramente todos los elementos de su modelo. Las **tasas o probabilidades de transición** deben expresarlas de manera general (**forma implícita**).
> **d.** Suponga ahora que cada vez que llega una solicitud de alquiler, esta es **exitosa** (se alquila el vehículo) con **probabilidad $p_e$**. Modifique su modelo del numeral anterior para capturar esta modificación. Exprese claramente todos los elementos de su modelo. Las **tasas o probabilidades de transición** deben expresarlas de manera general (**forma implícita**).

## a. Provea una expresión que permita calcular la **probabilidad** de que en **un día** de operación **lleguen menos de 5** solicitudes de alquiler.

**Relacionado:** [[Distribucion de Poisson]] (Menos de $k$ llegadas)


$$
P(X<k)=\sum_{i=0}^{k-1}\frac{(\lambda t)^{i}e^{-\lambda t}}{i!}
$$
$$
P(N(24)<5)=\sum_{i=0}^{4}\frac{( \lambda*24)^{i}e^{-\lambda *24}}{i!}
$$


$$
P\{N(24)<5\} =
e^{-\lambda\cdot 24}
\Bigg[
\frac{(\lambda\cdot 24)^0}{0!}+
\frac{(\lambda\cdot 24)^1}{1!}+
\frac{(\lambda\cdot 24)^2}{2!}+
\frac{(\lambda\cdot 24)^3}{3!}+
\frac{(\lambda\cdot 24)^4}{4!}
\Bigg].
$$

## b. Provea una expresión que permita calcular la probabilidad de que el tiempo entre la primera y la segunda solicitud de alquiler sea menor a una hora.

**Relacionado:** [[Distribucion Exponencial]] (Funcion de distribucion acumulada)


$$
P(T\leq 1)=1-e^{-\lambda}
$$
## **c.** Modele este sistema empleando una **cadena de Markov**. Exprese claramente todos los elementos de su modelo. Las **tasas o probabilidades de transición** deben expresarlas de manera general (**forma implícita**).

### Parametros
- Solicitudes de alquiler segun **Poisson** con tasa $\lambda$ (por **hora**)
- Duracion de un alquiler **exponencial** con **media** $m$ dias
$$
\mu=\frac{1}{m}\text{ dias}=\frac{24}{m}\text{ horas}
$$
### Variable
$$
X_{n}:\text{ Numero de vehiculos alquilados en el tiempo } t
$$
### Espacio de estado
$$
S_{x}=\{0,1,2,3,\dots,50\}
$$
### Matriz generadora
- La exponenciales compiten por terminar primero (n=i)
$$
\begin{bmatrix}
&0&1&2&3&\dots&50 \\
0&-\lambda&\lambda&0&0&&0 \\
1&n\mu&-(n\mu+\lambda)&\lambda&0&&0 \\
2&0&n\mu&-(n\mu+\lambda)&\lambda&&0 \\
3&0&0&n\mu&\dots \\
\dots \\
50&
\end{bmatrix}
$$

$$
\begin{cases}
\lambda&j=i+1,i<50 \\
n\mu&j=i-1,j>0,n=i\\
0&d.l.c
\end{cases}

$$


In [3]:
# Importar las librerías necesarias 
import numpy as np  
from jmarkov.ctmc import ctmc


from scipy.stats import poisson 
from scipy.stats import expon
from scipy.stats import binom

In [6]:
# Parametros
lambda_alquiler =7
m = 12
N = 50

## a. Provea una expresión que permita calcular la **probabilidad** de que en **un día** de operación **lleguen menos de 5** solicitudes de alquiler.

In [8]:
probability = poisson.cdf(4, lambda_alquiler*24)
print(f"Probabilidad de que un auto no sea alquilado en 24 horas: {probability}")

Probabilidad de que un auto no sea alquilado en 24 horas: 3.7149636358541743e-66


## b. Provea una expresión que permita calcular la probabilidad de que el tiempo entre la primera y la segunda solicitud de alquiler sea menor a una hora.

In [16]:
probability = expon.cdf(1, scale=1/lambda_alquiler)
print(f"Probabilidad de que un auto sea alquilado en la primera hora: {probability}")

Probabilidad de que un auto sea alquilado en la primera hora: 0.9990881180344455


## c. Modele este sistema empleando una **cadena de Markov**. Exprese claramente todos los elementos de su modelo. Las **tasas o probabilidades de transición** deben expresarlas de manera general (**forma implícita**).

In [19]:
# Tasa de alquiler exponencial diario
mu = (1/m)*24

estados = np.array([i for i in range(0,N+1)])
matriz_transicion = np.zeros((len(estados), len(estados)))

for i in range(len(estados)):
    for j in range(len(estados)):
        n = 0
        if j == i + 1 and i < N:
            matriz_transicion[i, j] = lambda_alquiler
        elif j == i - 1 and j > 0:
            n = i
            matriz_transicion[i, j] = n * mu
        else:
            matriz_transicion[i, j] = 0
    matriz_transicion[i, i] = -np.sum(matriz_transicion[i])
    
np.round(matriz_transicion,4)
            

array([[  -7.,    7.,    0., ...,    0.,    0.,    0.],
       [   0.,   -7.,    7., ...,    0.,    0.,    0.],
       [   0.,    4.,  -11., ...,    0.,    0.,    0.],
       ...,
       [   0.,    0.,    0., ..., -103.,    7.,    0.],
       [   0.,    0.,    0., ...,   98., -105.,    7.],
       [   0.,    0.,    0., ...,    0.,  100., -100.]])

In [21]:
cadena = ctmc(matriz_transicion, estados)